# Problem 1 (Practice)

Consider the integral
\begin{align*}
  I = \int_0^2 \sin^2\left[\frac{1}{x(2-x)}\right] dx.
\end{align*}

(a) Write a program to evaluate the integral using the "hit-or-miss" Monte Carlo method with 100 sample points. Try the evaluation 1,000 times and print out the mean and standard deviation using np.mean and np.std. Considering that the true value of the integral is 1.4514015, what is the percentage error of the mean? (20 pts)

(b) Apply the same algorithm for $N=10^2,10^3,10^4,10^5$ with 1,000 iterations each. Plot the standard deviation as a function of $N$, in log-log scale. How does the standard deviation scale with $N$? Does this confirm your expectations of the scaling?

(40 pts)

(c) Now repeat (b) with the mean value method. Is it more or less precise than the hit-or-miss Monte Carlo method? Does the scaling change? (40 pts)

---
# Problem 2

Calculate a value for the integral
\begin{align*}
  I = \int_0^1 \frac{x^{-1/2}}{e^x+1}dx
\end{align*}
using the importance sampling formula in the lecture notes, as follows.

(a) Plot the integrand in the domain. Is there a divergence and where? (10 pts)

(b) Use the mean value method to evaluate the integral 10,000 times, for $N=100$. What is the mean and standard deviation, using np.mean and np.std (Run the entire code a couple of times and present a reasonable standard deviation). (20 pts)

(c) Now let us use the importance sampling method to more correctly evaluate the integral. By inspection, the probability density function $p(x)=Ax^{-1/2}$ should be a good candidate. What is $A$? (10 pts)

(d) We want to sample from $p(x)$ given a uniformly distributed variable $\eta_i$ as $x_i = s(\eta_i)$. What is $s(\eta)$? (20 pts)

(e) Write a code the uses 100 random samples from $p(x)$ and use this to calculate $I$ 10,000 times, as in (b). What is the mean and the standard deviation? Is it better than the mean value method? (40 pts)